In [1]:
import pandas as pd
from scipy import stats as st
from scipy.stats import chi2, chi2_contingency

In [2]:
# Предположим мы получаем выгрузки из системы в формате csv. Проведем некий первичный анализ полученных данных...

users = pd.read_csv('./datasets/users.csv')
calls = pd.read_csv('./datasets/calls.csv', sep=';')
orders = pd.read_csv('./datasets/orders.csv')

Расшифровка статуса заказа:
    N - заказ завершился успешно без проблем
    R - заказ был отменен службой поддержки по причине не корректного адреса
    M - заказ был отменен по другой причине

In [3]:
df_files = [
    {"df": users, "name": "Пользователи", "data_column": ["reg_data"]},
    {"df": calls, "name": "Звонки", "data_column": ["call_data"]},
    {"df": orders, "name": "Заказы", "data_column": ["order_data"]},
]

In [4]:
def dfs_info_and_header(dfs: dict):
    for df in dfs:
        print(df["name"])
        print("-" * 15)
        display(df["df"].info())
        display(df["df"].head())

Посмотрим, какие данные до нас добрались

In [5]:
dfs_info_and_header(df_files)

Пользователи
---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   user_id        6 non-null      int64 
 1   device         6 non-null      object
 2   reg_data       6 non-null      object
 3   group_of_test  6 non-null      object
dtypes: int64(1), object(3)
memory usage: 320.0+ bytes


None

,user_id,device,reg_data,group_of_test
0,1,IOS,2020-01-05,A
1,2,Android,2020-01-05,B
2,3,desktop,2020-01-05,B
3,4,IOS,2020-01-06,A
4,5,IOS,2020-01-05,A


Звонки
---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     4 non-null      int64 
 1   response     4 non-null      object
 2   response_id  4 non-null      object
 3   call_data    4 non-null      object
dtypes: int64(1), object(3)
memory usage: 256.0+ bytes


None

,order_id,response,response_id,call_data
0,5,Created new order. Incorrect address,R,2020-05-06
1,8,Order rejected because ...,M,2020-05-07
2,10,Recreat,R,2020-05-06
3,12,Order rejected,M,2020-05-07


Заказы
---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     14 non-null     int64 
 1   order_id    14 non-null     int64 
 2   status      14 non-null     object
 3   order_data  14 non-null     object
dtypes: int64(2), object(2)
memory usage: 576.0+ bytes


None

,user_id,order_id,status,order_data
0,1,1,N,2020-05-05
1,1,2,N,2020-05-05
2,1,3,N,2020-05-06
3,2,4,N,2020-05-05
4,2,5,R,2020-05-06


In [6]:
def checkout_duplicated(dfs: dict):
    for df in dfs:
        print("-" * 15)
        print(df["name"])
        print("Число дубликатов:", df["df"].duplicated().sum())

Проверим на дубликаты в выгрузке, на всякий случай.

In [7]:
checkout_duplicated(df_files)

---------------
Пользователи
Число дубликатов: 0
---------------
Звонки
Число дубликатов: 0
---------------
Заказы
Число дубликатов: 0


In [8]:
def convert_data_column(dfs: dict, date_format: str):
    for df_t in dfs:
        if 'data_column' in df_t:
            df = df_t["df"]
            for data_column in df_t['data_column']:
                df[data_column] = pd.to_datetime(df[data_column], format=date_format)

Конвертируем в паднасовский формат время

In [9]:
convert_data_column(df_files, "%Y-%m-%d")

In [10]:
users['group_of_test'] = users['group_of_test'].astype("category")

После такой предварительно подготовке, разделим на группы. Уберем не участвующих в тесте. Посмотрим

In [11]:
df_A = users[users["group_of_test"] == "A"]
df_B = users[users["group_of_test"] == "B"]

In [12]:
df_A = pd.merge(pd.merge(df_A, orders, on="user_id"), calls, on="order_id", how="left")

In [13]:
df_B = pd.merge(pd.merge(df_B, orders, on="user_id"), calls, on="order_id", how="left")

In [14]:
df_B

,user_id,device,reg_data,group_of_test,order_id,status,order_data,response,response_id,call_data
0,2,Android,2020-01-05,B,4,N,2020-05-05,NaN,NaN,NaT
1,2,Android,2020-01-05,B,5,R,2020-05-06,Created new order. Incorrect address,R,2020-05-06
2,2,Android,2020-01-05,B,6,N,2020-05-06,NaN,NaN,NaT
3,3,desktop,2020-01-05,B,7,N,2020-05-05,NaN,NaN,NaT
4,3,desktop,2020-01-05,B,8,M,2020-05-07,Order rejected because ...,M,2020-05-07
5,6,IOS,2020-01-06,B,14,N,2020-05-07,NaN,NaN,NaT


Начнем сверять метрики

In [15]:
def number_of_rejected_orders(df: pd.DataFrame):
    nro_to_all = df[df["status"] != "N"].shape[0] / df.shape[0]
    nro_incorrect_order = df[df["status"] == "R"].shape[0] / df[df["status"] != "N"].shape[0]
    nro_conversion = df[df["status"] == "N"].shape[0] / df.shape[0]
    return nro_to_all, nro_incorrect_order, nro_conversion

In [16]:
def call_reorder(df: pd.DataFrame):
    count_call = df.count()["response_id"]
    count_cr = df[df["response_id"] == "R"].shape[0]
    percent_cr = count_cr / count_call
    return count_call, count_cr, percent_cr

In [17]:
def conversion(df: pd.DataFrame, old_data):
    sold = df[df["status"] == "N"].shape[0]
    conversion_rate = sold / df.shape[0] - old_data["cr"]
    cr = df_A.groupby(['user_id']).size().reset_index(name='count')
    repeat_rate = len(cr[cr['count'] != 1]) / df.shape[0] - old_data["rr"]
    return conversion_rate, repeat_rate

In [18]:
def statistics(df: pd.DataFrame):
    print("Число отмененных заказов в общем число заказов {:0.3%}.\n"
          "Процент отмененных заказов по причине некорректного адреса, {:0.2%} от всех отмененых заказов.\n"
          "Число завершенных заказов в общем числе заказов {:0.3%}.\n"
          .format(*number_of_rejected_orders(df)))
    print("Число звонков {:d}.\n"
          "число звонков с целью поменять адрес {:d}.\n"
          "Процент звонков с целью поменять адрес {:0.2%}.\n"
          .format(*call_reorder(df)))
    old_data = {
        "cr": 0.5
        , "rr": 0.2
    }
    print("Изменение числа завершенных заказов по отношению к предыдущему {:+0.2f}pp.\n"
          "Изменение числа повторных обращений к предыдущему {:+0.2f}pp.\n"
          .format(*conversion(df, old_data)))

In [19]:
statistics(df_A)

Число отмененных заказов в общем число заказов 25.000%.
Процент отмененных заказов по причине некорректного адреса, 50.00% от всех отмененых заказов.
Число завершенных заказов в общем числе заказов 75.000%.

Число звонков 2.
число звонков с целью поменять адрес 1.
Процент звонков с целью поменять адрес 50.00%.

Изменение числа завершенных заказов по отношению к предыдущему +0.25pp.
Изменение числа повторных обращений к предыдущему +0.05pp.



In [20]:
statistics(df_B)

Число отмененных заказов в общем число заказов 33.333%.
Процент отмененных заказов по причине некорректного адреса, 50.00% от всех отмененых заказов.
Число завершенных заказов в общем числе заказов 66.667%.

Число звонков 2.
число звонков с целью поменять адрес 1.
Процент звонков с целью поменять адрес 50.00%.

Изменение числа завершенных заказов по отношению к предыдущему +0.17pp.
Изменение числа повторных обращений к предыдущему +0.13pp.



In [23]:
def chi_test(
        df_A: pd.DataFrame
        , df_B: pd.DataFrame
        , acceptance_criteria: float
):
    df = pd.concat([df_A, df_B])
    df = df[(df["status"] == "N") | (df["status"] == "R")]

    observed_values = pd.crosstab(df['group_of_test'], df['status']).values

    null_hyp = "Изменений в пользовательском поведении нет"
    alter_hyp = "Есть изменения в поведении пользователя"

    chi2_statistic, p_value, dof, expected_values = chi2_contingency(observed_values, correction=False)
    critical_value = chi2.ppf(1 - acceptance_criteria, dof)

    if chi2_statistic >= critical_value:
        print(
            "Так как chi-square статистика {:.3f} выше критического значения {:.3f}".format(chi2_statistic,critical_value),
            f" - мы отвергаем нулевую гипотезу и заявляем: {alter_hyp}.")
    else:
        print(
            "Так как chi-square статистика {:3f} ниже критического значения {:3f}".format(chi2_statistic,critical_value),
            f" - мы не отвергаем нулевую гипотезу и заявляем: {null_hyp}.")
    if p_value <= acceptance_criteria:
        print(
            "Так как p_value {:.3f} ниже допустимой погрешности {:.3f}".format(p_value, acceptance_criteria),
            f" - мы отвергаем нулевую гипотезу и заявляем: {alter_hyp}.")
    else:
        print(
            "As our p_value of {:.3f} выше допустимой погрешности {:.3f}".format(p_value, acceptance_criteria),
            f" - мы не отвергаем нулевую гипотезу и заявляем: {null_hyp}.")

In [24]:
chi_test(df_A, df_B,0.05)

Так как chi-square статистика 0.068571 ниже критического значения 3.841459  - мы не отвергаем нулевую гипотезу и заявляем: Изменений в пользовательском поведении нет.
As our p_value of 0.793 выше допустимой погрешности 0.050  - мы не отвергаем нулевую гипотезу и заявляем: Изменений в пользовательском поведении нет.
